In [1]:
# Libraries 
import numpy as np 
import pandas as pd 

In [7]:
main_path = 'course-material-time-series-forecasting-product'
nyc_data = main_path+str("/nyc_data.csv")
print(nyc_data)

course-material-time-series-forecasting-product/nyc_data.csv


In [10]:
# Best practice when dealing with Time series: Date variable goes to index
# To do so, inform 'index_col='
# To write the time in a 'standard format', which is YYYY-MM-DD, inform 'parse_dates='
# This df is very big. To only show a small snipped, inform 'df.head()'
df = pd.read_csv(nyc_data, index_col=0, parse_dates=True)
df.head() 

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [11]:
# Rename variable 
# Let's rename the dependent variable. 
# (Which is the one that we want to predict)
df = df.rename(columns={'Demand': 'y'})
df.head()

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077
